# 🍽️ YOLOv8 - Food Classification (Using Our First Pretrained Model)

**YOLOv8** is an advanced CNN-based model used for object detection and classification.  
We'll use its **classification head** (`yolov8n-cls.pt`), pretrained on ImageNet,  
to fine-tune on a **subset of the Food-101 dataset** with 5 classes for high-speed, high-accuracy food recognition.

### 🎯 Goal  
Build a production-grade food classification system using YOLOv8’s classification head.  
We’ll fine-tune it on the following **5 food classes**:
- pizza  
- grilled_chicken  
- sushi  
- ice_cream  
- hamburger  

We'll use **advanced training techniques** to boost performance.

### ⚙️ Techniques Used  
- **Albumentations**: Advanced image augmentation for better generalization  
- **Cosine Annealing LR**: Smooth learning rate decay  
- **Model Ensembling** *(optional)*: Combines predictions for higher accuracy  


### 📁 Directory Structure  
```
📦 Project Root
├── Food_Classification_YOLOv8_Module12.ipynb
├── generate_yaml.py
├── food-101/
│   ├── images/
│   │   ├── pizza/
│   │   ├── grilled_chicken/
│   │   ├── sushi/
│   │   ├── ice_cream/
│   │   ├── hamburger/
│   ├── meta/
│   └── food101.yaml
```

### 📦 Install Requirements  
Make sure to install all dependencies before running the code:
```bash
pip install torch==2.4.1 torchvision==0.19.1
pip install opencv-python-headless==4.10.0
pip install ultralytics==8.2.28
pip install albumentations==1.4.8
```

### 🔧 Basic Setup (with PyTorch + YOLOv8)

In [ ]:
# I generally use PyTorch so I can understand exactly what’s happening

import torch 
import torch.nn as nn 
import torch.optim as optim 
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, Subset 
import cv2
import numpy as np
import logging 
import os 
import albumentations as A
from typing import List, Optional, Dict, Tuple


In [ ]:
# logging setup  to track executions and errors

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers = [
                        logging.StreamHandler(), #to show logs in console 
                        logging.FileHandler('yolo_training.log') # save logs to a file
                    ])

logger = logging.getLogger(__name__)

In [ ]:
# configurations class for project settings

class Config:
    def __init__(self):
        self.data_dir = "./foof/food-101"
        self.classes = ['pizza','grilled_chicken','sushi','ice_cream','hamburger']
        self.max_images_per_class = 100
        self.test_images_per_class = 20
        self.epochs = 18
        self.batch_size = 5 # batch size optimized for gpu 
        self.img_size = 224 # image size for yolo v8 
        self.mode_path_nano = "yolov8n_food_classifier.pt" #nano model save path 
        self.model_path_small = "yolov8s_food_classifier.pt" #small model save path 
        self.yaml_path = os.path.join(self.data_dir, 'food101.yaml')
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.lr = 0.001
        logging.info(f"Using device : {self.device}")
